In [6]:
"""
Description: Get current currency rates from ECB.
Author: Richard van Wijk
Maintainer: Richard van Wijk
Email: vanwijk@ducata.com
Status: Dev
"""
from currency_converter import CurrencyConverter
from datetime import timedelta, date, datetime
import pandas as pd 

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

c = CurrencyConverter('http://www.ecb.int/stats/eurofxref/eurofxref-hist.zip', fallback_on_wrong_date=True, fallback_on_missing_rate=True,decimal=True)

start_date = date(2000, 1, 1)
#end_date = date(2006,12,31)
end_date = date.today()

dateList = []

for single_date in daterange(start_date, end_date):

    curList = []
    curList.append(single_date)
    
    for currency in c.currencies:
        
        value = c.convert(1, 'EUR', currency, date=single_date)
        curList.append(float(value))
   
    dateList.append(curList)

columnsList = ['date'] + list(c.currencies)

df = pd.DataFrame(dateList, columns = columnsList)
df = df.sort_values(by="date", ascending=False)

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 43)
pd.set_option('display.width', 800)
pd.set_option('display.max_colwidth', 15)
pd.set_option('display.precision', 3)

display(df)

,date,KRW,GBP,NOK,MYR,SKK,DKK,MTL,CZK,IDR,BGN,AUD,LVL,CHF,BRL,CNY,SIT,INR,PHP,EEK,THB,LTL,TRL,EUR,HRK,HUF,SEK,RON,SGD,ZAR,USD,MXN,TRY,ROL,NZD,JPY,ISK,ILS,PLN,CAD,HKD,CYP,RUB
7592,2020-10-14,1346.700,0.904,10.822,4.877,30.126,7.444,0.429,27.313,17311.280,1.956,1.638,0.703,1.074,6.543,7.909,239.640,86.100,57.244,15.647,36.648,3.453,1.836e+06,1.0,7.579,363.490,10.365,4.874,1.595,19.390,1.175,25.108,9.333,36030.000,1.763,123.700,163.200,3.973,4.507,1.543,9.106,0.585,90.933
7591,2020-10-13,1351.000,0.906,10.792,4.882,30.126,7.443,0.429,27.356,17403.270,1.956,1.640,0.703,1.073,6.539,7.939,239.640,86.425,57.325,15.647,36.676,3.453,1.836e+06,1.0,7.579,360.200,10.343,4.875,1.601,19.445,1.179,25.018,9.316,36030.000,1.772,124.380,163.200,3.990,4.485,1.545,9.135,0.585,90.804
7590,2020-10-12,1355.300,0.906,10.793,4.893,30.126,7.443,0.429,27.177,17378.690,1.956,1.636,0.703,1.074,6.527,7.958,239.640,86.476,57.268,15.647,36.754,3.453,1.836e+06,1.0,7.571,356.830,10.393,4.873,1.601,19.481,1.180,25.073,9.282,36030.000,1.774,124.410,163.200,3.993,4.481,1.549,9.144,0.585,90.765
7589,2020-10-11,1353.707,0.908,10.816,4.889,30.126,7.443,0.429,27.155,17365.797,1.956,1.637,0.703,1.075,6.544,7.940,239.640,86.383,57.206,15.647,36.712,3.453,1.836e+06,1.0,7.573,356.647,10.402,4.872,1.600,19.465,1.180,25.076,9.297,36030.000,1.777,124.590,163.067,3.991,4.477,1.551,9.143,0.585,90.824
7588,2020-10-10,1352.113,0.910,10.839,4.885,30.126,7.442,0.429,27.132,17352.903,1.956,1.639,0.703,1.076,6.562,7.922,239.640,86.291,57.145,15.647,36.671,3.453,1.836e+06,1.0,7.575,356.463,10.411,4.872,1.599,19.448,1.180,25.080,9.313,36030.000,1.780,124.770,162.933,3.988,4.474,1.553,9.142,0.585,90.883
7587,2020-10-09,1350.520,0.912,10.862,4.881,30.126,7.442,0.429,27.110,17340.010,1.956,1.641,0.703,1.077,6.580,7.905,239.640,86.198,57.083,15.647,36.629,3.453,1.836e+06,1.0,7.577,356.280,10.420,4.872,1.599,19.431,1.179,25.083,9.328,36030.000,1.782,124.950,162.800,3.986,4.470,1.554,9.141,0.585,90.941
7586,2020-10-08,1355.480,0.910,10.904,4.885,30.126,7.442,0.429,27.092,17362.200,1.956,1.642,0.703,1.080,6.583,7.989,239.640,86.186,57.004,15.647,36.724,3.453,1.836e+06,1.0,7.568,357.480,10.447,4.876,1.598,19.500,1.177,25.142,9.330,36030.000,1.786,124.690,162.800,3.991,4.484,1.558,9.118,0.585,91.067
7585,2020-10-07,1362.700,0.914,10.931,4.892,30.126,7.442,0.429,27.064,17359.510,1.956,1.650,0.703,1.079,6.545,7.993,239.640,86.263,56.979,15.647,36.758,3.453,1.836e+06,1.0,7.569,359.250,10.483,4.875,1.599,19.579,1.177,25.261,9.272,36030.000,1.786,124.730,162.800,4.004,4.486,1.564,9.122,0.585,91.972
7584,2020-10-06,1368.490,0.911,10.860,4.899,30.126,7.441,0.429,27.035,17379.930,1.956,1.647,0.703,1.078,6.536,8.011,239.640,86.572,57.137,15.647,36.795,3.453,1.836e+06,1.0,7.569,359.720,10.514,4.876,1.603,19.453,1.179,25.163,9.162,36030.000,1.773,124.580,162.800,4.019,4.487,1.563,9.141,0.585,91.772
7583,2020-10-05,1363.720,0.908,10.886,4.887,30.126,7.440,0.429,27.096,17324.610,1.956,1.638,0.703,1.078,6.657,8.013,239.640,86.087,56.961,15.647,36.846,3.453,1.836e+06,1.0,7.568,358.170,10.464,4.874,1.601,19.345,1.177,25.255,9.161,36030.000,1.770,124.250,162.400,4.024,4.498,1.561,9.120,0.585,92.535
